In [1]:
# imports
import keras
import numpy as np
import pandas as pd
import torch_cnn as tcnn
from keras_cnn import keras_CNN as kcnn
import data_processing as d
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split   

In [2]:
# Data retieval
X, y = d.get_encoded_data()
X_train, X_test, y_train, y_test = d.splitData(X, y)

# Keras CNN Data
X_train = X_train.reshape(1, X_train.shape[0], X_train.shape[1])
y_train = y_train.reshape(1, y_train.shape[0], y_train.shape[1])
X_test = X_test.reshape(1, X_test.shape[0], X_test.shape[1])
y_test = y_test.reshape(1, y_test.shape[0], y_test.shape[1])
    
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)


# Convert data to Tensors to Incease Computation time for Torch CNN
X_tensor_train,Y_tensor_train  = d.toTensors(X, y)

# Change Y data for SVM
svm_X_train, svm_X_test, svm_y_train, svm_y_test =  train_test_split(X, d.get_SVM_y(), test_size=0.33, random_state=1)

In [3]:
# Variables

EPOCHS = 1000 # arbitrary

# Keras CNN
KERNEL_SIZE = 3
FILTERS = 128
DROPOUT_RATE = 0.25

# Torch CNN
N_FEATURES = X_train.shape[1] # Number of features for the input layer
NUM_ROWS_TRAINING = X_train.shape[0] # Number of rows
N_HIDDEN = N_FEATURES * 10 # Size of first linear layer
N_CNN_KERNEL, MAX_POOL_KERNEL = 3, 4 # CNN kernel size

In [4]:
# Keras CNN
#Build Model
cnn1D = kcnn(6, EPOCHS)
#new_X_train, new_X_test, new_y_train, new_y_test = cnn1D.reshape_and_chng_type(X_train, X_test, y_train, y_test)
model = cnn1D.getModel()
optimizer = keras.optimizers.Adam(learning_rate=1e-5, decay=5e-4)

model.compile(optimizer, 'mse', metrics=[keras.metrics.Accuracy()])

history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    verbose=1,
    epochs=500
)

# for metric in history.history:
#     cnn1D.plot(history.history, EPOCHS, metric)

print("Evaluate on test data")
results = model.predict(X_test, batch_size=64)
pred_y = d.standarize_predictions(results.reshape(results.shape[1], results.shape[2]))
print(results)
print("test acc:", accuracy_score(pred_y, y_test.reshape(y_test.shape[1], y_test.shape[2])))

Epoch 1/500
1/1 [==============================] - 1s 780ms/step - loss: 0.1335 - accuracy: 0.8248
Epoch 2/500
1/1 [==============================] - 0s 74ms/step - loss: 0.1335 - accuracy: 0.8250
Epoch 3/500
1/1 [==============================] - 0s 78ms/step - loss: 0.1330 - accuracy: 0.8252
Epoch 4/500
1/1 [==============================] - 0s 81ms/step - loss: 0.1321 - accuracy: 0.8269
Epoch 5/500
1/1 [==============================] - 0s 78ms/step - loss: 0.1318 - accuracy: 0.8289
Epoch 6/500
1/1 [==============================] - 0s 85ms/step - loss: 0.1331 - accuracy: 0.8256
Epoch 7/500
1/1 [==============================] - 0s 85ms/step - loss: 0.1320 - accuracy: 0.8263
Epoch 8/500
1/1 [==============================] - 0s 89ms/step - loss: 0.1328 - accuracy: 0.8263
Epoch 9/500
1/1 [==============================] - 0s 83ms/step - loss: 0.1322 - accuracy: 0.8257
Epoch 10/500
1/1 [==============================] - 0s 94ms/step - loss: 0.1319 - accuracy: 0.8268
Epoch 11/500
1/1 [

In [ ]:
### Torch CNN
# Build CNN
net = tcnn.CNN(n_feature=N_FEATURES, n_hidden=N_HIDDEN, n_output=13, n_cnn_kernel=N_CNN_KERNEL)   # define the network    
# Train and Test
train_results, results = tcnn.trainTestCNN(net,X_tensor_train,Y_tensor_train)
# Plotting
for metric in train_results:
    kcnn.plot(train_results, EPOCHS, metric, "Training")
for metric in results:
    kcnn.plot(results, EPOCHS/100, metric, "Testing")

In [ ]:
# SVM
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
accuracy = []
# Train and Test
for x in range(0,EPOCHS):
    clf.fit(svm_X_train,  svm_y_train)
    if x % 100 == 0:
        predictions=clf.predict(svm_X_test)
        accuracy.append(accuracy_score(svm_y_test, predictions))
# Plotting        
results = {'Accuracy':accuracy}
for metric in results:
    kcnn.plot(results, EPOCHS/100, metric, "Testing")        

In [ ]:
df = pd.read_csv('datasets/VirusSample.csv')
df = df.drop(columns=['file'])
pd.set_option('max_colwidth', 150)
shuffle(df).head(20)